In [1]:
#import necessary packages
import pandas as pd 
import splitfolders

from keras.preprocessing.image import ImageDataGenerator

from keras import models
from keras import layers
from keras import optimizers
from keras import metrics

Using TensorFlow backend.


In [2]:
#use the splitfolders package to split our image data into train, validation, and test sets.
#this is now commented out so that we don't create a new folder each time we run the notebook

#splitfolders.ratio("Data", output="Data_Split",
#    seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

In [3]:
#create DataGenerator objects for each set with rescaling for maximum pixel value of 255
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [33]:
#create a train_generator from the training directory that has been created from the split folders package
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'Data_Split/train',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=4280,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary', 
        color_mode='grayscale')


Found 4280 images belonging to 2 classes.


In [34]:
#create a validation_generator from the training directory that has been created from the split folders package
validation_generator = val_datagen.flow_from_directory('Data_Split/val',
                                                        target_size=(150, 150),
                                                        batch_size=978,
                                                        class_mode='binary',
                                                        color_mode='grayscale')

Found 978 images belonging to 2 classes.


In [35]:
#create a test_generator from the training directory that has been created from the split folders package
test_generator = test_datagen.flow_from_directory('Data_Split/test',
                                                  target_size=(150, 150),
                                                  batch_size=1173,
                                                  class_mode='binary',
                                                  color_mode='grayscale')

Found 1173 images belonging to 2 classes.


In [48]:
train_data, train_labels = next(train_generator)

In [49]:
train_data.shape

(4280, 150, 150, 1)

In [50]:
train_labels.shape

(4280,)

In [51]:
train_data = train_data.reshape(train_data.shape[0], -1)

In [52]:
train_data.shape

(4280, 22500)

In [53]:
baseline_model = models.Sequential()

In [54]:
baseline_model.add(layers.Dense(12, activation='relu', input_shape=(22500,)))
baseline_model.add(layers.Dense(1, activation='sigmoid'))

In [55]:
baseline_model.compile(optimizer='SGD', 
                       loss='binary_crossentropy',
                       metrics=['accuracy', metrics.Precision(), metrics.Recall()])

In [56]:
baseline_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 12)                270012    
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 13        
Total params: 270,025
Trainable params: 270,025
Non-trainable params: 0
_________________________________________________________________


In [65]:
baseline_fit = baseline_model.fit(train_data,
                                  train_labels,
                                  epochs=5,
                                  batch_size=32,
                                  #validation_data=validation_generator,
                                  verbose=1)

Epoch 1/5
4280/4280 [==============================] - 1s 144us/step - loss: 0.1680 - accuracy: 0.9460 - precision_6: 0.9578 - recall_6: 0.9721
Epoch 2/5
4280/4280 [==============================] - 1s 142us/step - loss: 0.1680 - accuracy: 0.9456 - precision_6: 0.9591 - recall_6: 0.9700
Epoch 3/5
4280/4280 [==============================] - 1s 142us/step - loss: 0.1665 - accuracy: 0.9439 - precision_6: 0.9596 - recall_6: 0.9672
Epoch 4/5
4280/4280 [==============================] - 1s 153us/step - loss: 0.1642 - accuracy: 0.9477 - precision_6: 0.9604 - recall_6: 0.9715
Epoch 5/5
4280/4280 [==============================] - 1s 176us/step - loss: 0.1617 - accuracy: 0.9467 - precision_6: 0.9600 - recall_6: 0.9706
